In [1]:
import pandas as pd
import os
import numpy as np

# Confirm the files are in the correct location
assert os.path.exists('data/train.csv')
assert os.path.exists('data/test.csv')

# Load the training data csvs
train_csv = pd.read_csv('data/train.csv')
test_csv = pd.read_csv('data/test.csv')
train_csv.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# Model / data parameters - used for reshaping
num_classes = 10
input_shape = (28, 28, 1)

# Convert the training table into x/y sets
train_ydata = pd.DataFrame(train_csv["label"], columns=["label"])
train_xdata = [row.to_numpy().reshape(input_shape) for index, row in train_csv.iloc[:, 1:].iterrows()]

# Create x set for test set (for submission)
test_xdata = [row.to_numpy().reshape(input_shape) for index, row in test_csv.iloc[1:, :].iterrows()]


In [6]:
from sklearn.model_selection import train_test_split

# Split data to train and validation sets
x_train, y_train, x_val, y_val = train_test_split(train_xdata, train_ydata, test_size=0.20, random_state=0)

# Normalize the pixel arrays 
x_train = [arr.astype(np.float32) / 255.0 for arr in x_train]
x_test = [arr.astype(np.float32) / 255.0 for arr in test_xdata]



In [4]:
import keras
from keras import layers

# Configure Model Structure
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

2022-11-06 17:11:54.279890: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-06 17:11:54.280053: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                             

In [11]:
# Train the model and evaluate
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

score = model.evaluate(x_val, y_val, verbose=1)
print("Validation loss:", score[0])
print("Validation accuracy:", score[1])

In [ ]:
# Evaluate the model over the submission dataset
y_test = model.predict(test_xdata, verbose=0)

# Save the data to a file
output_data = pd.DataFrame({"ImageId": np.arange(len(y_test)), "Label": y_test})
output_data.to_csv("prediction.csv")
